# Stokes 2D

Staggered-grid finite-difference implementation of the _Data-Assimilation Example from Glaciology_ given in [1].

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import numpy as np
from scipy import optimize
import cmocean
import cmocean.cm as cmo
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams["figure.dpi"] = 100

In [ ]:
# Parameters
alpha = 0.1/180*np.pi # inclination of the plane
height = 1.0e3        # height of the ice sheet (m)
length = 2.0e4        # length of the domain (m)
nx = 64               # number of grid points in x-direction
ny = 32               # number of grid points in y-direction, >3
rho = 910.0           # density of ice, kg m^-3
mu0 = 1.0             # initial viscosity
g = 9.81              # gravitational acceleration (m s^-2)
A_glen = 1.0e-16      # Glen flow law ice softness (Pa^-n a^-1)
n_glen = 3            # Glen flow law exponent
N = 30                # number of coefficients in the trigonometric expansion of beta^2

In [ ]:
# Setup the grid
hx = length/(nx-1)
hy = height/(ny-1)
x = np.linspace(0,length,nx)
y = np.linspace(0,height,ny)

In [ ]:
# Initialize coefficients
p = np.zeros(N+1); p[0] = 1000.0
p_desired = np.zeros(N+1); p_desired[0] = 1000.0; p_desired[1] = 1000.0
print('p_desired =', p_desired)

In [ ]:
# Compute the friction coefficient beta^2
def beta_squared(p):
    res = p[0]
    for k in range(1, N//2+1):
        res += p[2*k-1]*np.sin(2*np.pi*k*x/length)
        res += p[2*k]*np.cos(2*np.pi*k*x/length)
    return res

plt.figure(1)
fig, ax = plt.subplots()
ax.plot(x, beta_squared(p),label="initial")
plt.plot(x, beta_squared(p_desired), label="desired")
plt.xlim([0, length])
legend = ax.legend()
plt.xlabel(r'$x$')
plt.ylabel(r'$\beta^2$', rotation=0)
plt.show()

In [ ]:
# Compute current beta^2
beta2 = beta_squared(p)

In [ ]:
# Define node indices
node_indices = np.arange((nx-1)*(ny-1)).reshape((ny-1, nx-1), order="F")

In [ ]:
# Define indexing functions

def node_index(i, j):
    return j*(ny-1)+i

def u_index(i, j):
    n = 3*(nx-1)*(ny-1)
    return (3*node_index(i, j))%n

def v_index(i, j):
    n = 3*(nx-1)*(ny-1)
    return (3*node_index(i, j) + 1)%n

def p_index(i, j):
    n = 3*(nx-1)*(ny-1)
    return (3*node_index(i, j) + 2)%n

In [ ]:
# Print info of matrix construction
def print_info(node, index):
    if verbose==True:
        if index%3==0:
            print("node {:<2d} -> index {}: Adding x-Stokes equation".format(node, 3*node))
        if index%3==1:
            print("node {:<2d} -> index {}: Adding y-Stokes equation".format(node, 3*node+1))
        if index%3==2:
            print("node {:<2d} -> index {}: Adding continuity equation".format(node, 3*node+2))

In [ ]:
# Construct the coefficient matrix A and the right-hand side vector b
verbose = False
A = np.zeros((3*(nx-1)*(ny-1),3*(nx-1)*(ny-1)))
b = np.zeros(3*(nx-1)*(ny-1))
for j in range(nx-1):
    for i in range(ny-1):
        node = node_index(i, j)
        
        # Add x-Stokes equations to surface nodes
        if node in node_indices[0,:]:
            print_info(node, 3*node)
            A[u_index(i,j),u_index(i,j+1)] = 2/hx**2
            A[u_index(i,j),u_index(i,j)] -= 2/hx**2
            A[u_index(i,j),u_index(i,j)] -= 2/hx**2
            A[u_index(i,j),u_index(i,j-1)] = 2/hx**2
            A[u_index(i,j),p_index(i,j)] = -1/hx
            A[u_index(i,j),p_index(i,j-1)] = 1/hx
            # A[u_index(i,j),u_index(i-1,j)] = 0 #1/hy**2 # ghost point
            # A[u_index(i,j),u_index(i,j)] -= 1/hy**2
            # A[u_index(i,j),v_index(i,j)] = 1/(hx*hy)
            #A[u_index(i,j),v_index(i,j-1)] = -1/(hx*hy)
            A[u_index(i,j),u_index(i,j)] -= 1/hy**2
            A[u_index(i,j),u_index(i+1,j)] = 1/hy**2
            A[u_index(i,j),v_index(i+1,j)] = -1/(hx*hy)
            A[u_index(i,j),v_index(i+1,j-1)] = 1/(hx*hy)
            
            b[u_index(i,j)] = -rho*g*np.sin(alpha)
            
        # Add y-Stokes equations to surface nodes
        if node in node_indices[0,:]:
            print_info(node, 3*node+1)
            # A[v_index(i,j),v_index(i-1,j)] = 2/hy**2
            # A[v_index(i,j),v_index(i,j)] -= 2/hy**2
            A[v_index(i,j),v_index(i,j)] -= 2/hy**2
            A[v_index(i,j),v_index(i+1,j)] = 2/hy**2
            # A[v_index(i,j),p_index(i-1,j)] = -1/hy
            A[v_index(i,j),p_index(i,j)] = 1/hy
            # A[v_index(i,j),u_index(i-1,j+1)] = 1/(hx*hy)
            # A[v_index(i,j),u_index(i,j+1)] = -1/(hx*hy)
            # A[v_index(i,j),v_index(i,j+1)] = 1/hx**2
            # A[v_index(i,j),v_index(i,j)] -= 1/hx**2
            # A[v_index(i,j),u_index(i-1,j)] = -1/(hx*hy)
            # A[v_index(i,j),u_index(i,j)] = 1/(hx*hy)
            # A[v_index(i,j),v_index(i,j)] -= 1/hx**2
            #A[v_index(i,j),v_index(i,j-1)] = 1/hx**2

            b[v_index(i,j)] = rho*g*np.cos(alpha)
            
        # Add continuity equations to surface nodes
        if node in node_indices[0,:]:
            print_info(node, 3*node+2)
            A[p_index(i,j),u_index(i,j+1)] = 1/hx
            A[p_index(i,j),u_index(i,j)] = -1/hx
            A[p_index(i,j),v_index(i,j)] = 1/hy
            A[p_index(i,j),v_index(i+1,j)] = -1/hy
            
            b[p_index(i,j)] = 0
        
        # Add x-Stokes equations to inner nodes
        if node in node_indices[1:-1,:].flatten():
            print_info(node, 3*node)
            A[u_index(i,j),u_index(i,j+1)] = 2/hx**2
            A[u_index(i,j),u_index(i,j)] -= 2/hx**2
            A[u_index(i,j),u_index(i,j)] -= 2/hx**2
            A[u_index(i,j),u_index(i,j-1)] = 2/hx**2
            A[u_index(i,j),p_index(i,j)] = -1/hx
            A[u_index(i,j),p_index(i,j-1)] = 1/hx
            A[u_index(i,j),u_index(i-1,j)] = 1/hy**2
            A[u_index(i,j),u_index(i,j)] -= 1/hy**2
            A[u_index(i,j),v_index(i,j)] = 1/(hx*hy)
            A[u_index(i,j),v_index(i,j-1)] = -1/(hx*hy)
            A[u_index(i,j),u_index(i,j)] -= 1/hy**2
            A[u_index(i,j),u_index(i+1,j)] = 1/hy**2
            A[u_index(i,j),v_index(i+1,j)] = -1/(hx*hy)
            A[u_index(i,j),v_index(i+1,j-1)] = 1/(hx*hy)
            
            b[u_index(i,j)] = -rho*g*np.sin(alpha)
            
        # Add y-Stokes equations to inner nodes
        if node in node_indices[1:-1,:].flatten():
            print_info(node, 3*node+1)
            A[v_index(i,j),v_index(i-1,j)] = 2/hy**2
            A[v_index(i,j),v_index(i,j)] -= 2/hy**2
            A[v_index(i,j),v_index(i,j)] -= 2/hy**2
            A[v_index(i,j),v_index(i+1,j)] += 2/hy**2
            A[v_index(i,j),p_index(i-1,j)] = -1/hy
            A[v_index(i,j),p_index(i,j)] = 1/hy
            A[v_index(i,j),u_index(i-1,j+1)] = 1/(hx*hy)
            A[v_index(i,j),u_index(i,j+1)] = -1/(hx*hy)
            A[v_index(i,j),v_index(i,j+1)] = 1/hx**2
            A[v_index(i,j),v_index(i,j)] -= 1/hx**2
            A[v_index(i,j),u_index(i-1,j)] = -1/(hx*hy)
            A[v_index(i,j),u_index(i,j)] = 1/(hx*hy)
            A[v_index(i,j),v_index(i,j)] -= 1/hx**2
            A[v_index(i,j),v_index(i,j-1)] = 1/hx**2
            
            b[v_index(i,j)] = rho*g*np.cos(alpha)
            
        # Add continuity equations to inner nodes
        if node in node_indices[1:-1,:].flatten():
            print_info(node, 3*node+2)
            A[p_index(i,j),u_index(i,j+1)] = 1/hx
            A[p_index(i,j),u_index(i,j)] = -1/hx
            A[p_index(i,j),v_index(i,j)] = 1/hy
            A[p_index(i,j),v_index(i+1,j)] = -1/hy
            
            b[p_index(i,j)] = 0
            
        # Add x-Stokes equations to base nodes
        if node in node_indices[-1,:]:
            print_info(node, 3*node)
            A[u_index(i,j),u_index(i,j+1)] = 2/hx**2
            A[u_index(i,j),u_index(i,j)] -= 2/hx**2
            A[u_index(i,j),u_index(i,j)] -= 2/hx**2
            A[u_index(i,j),u_index(i,j-1)] = 2/hx**2
            A[u_index(i,j),p_index(i,j)] = -1/hx
            A[u_index(i,j),p_index(i,j-1)] = 1/hx
            A[u_index(i,j),u_index(i-1,j)] = 1/hy**2
            A[u_index(i,j),u_index(i,j)] -= 1/hy**2
            A[u_index(i,j),v_index(i,j)] = 1/(hx*hy)
            A[u_index(i,j),v_index(i,j-1)] = -1/(hx*hy)
            A[u_index(i,j),u_index(i,j)] -= 1/hy*beta2[j]
            # A[u_index(i,j),u_index(i+1,j)] = 1/hy**2
            # A[u_index(i,j),v_index(i+1,j)] = -1/(hx*hy)
            # A[u_index(i,j),v_index(i+1,j-1)] = 1/(hx*hy)
            
            b[u_index(i,j)] = -rho*g*np.sin(alpha)
            
        # Add y-Stokes equations to base nodes
        if node in node_indices[-1,:]:
            print_info(node, 3*node+1)
            A[v_index(i,j),v_index(i-1,j)] = 2/hy**2
            A[v_index(i,j),v_index(i,j)] -= 2/hy**2
            A[v_index(i,j),v_index(i,j)] -= 2/hy**2
            # A[v_index(i,j),v_index(i+1,j)] += 2/hy**2
            A[v_index(i,j),p_index(i-1,j)] = -1/hy
            A[v_index(i,j),p_index(i,j)] = 1/hy
            A[v_index(i,j),u_index(i-1,j+1)] = 1/(hx*hy)
            A[v_index(i,j),u_index(i,j+1)] = -1/(hx*hy)
            A[v_index(i,j),v_index(i,j+1)] = 1/hx**2
            A[v_index(i,j),v_index(i,j)] -= 1/hx**2
            A[v_index(i,j),u_index(i-1,j)] = -1/(hx*hy)
            A[v_index(i,j),u_index(i,j)] = 1/(hx*hy)
            A[v_index(i,j),v_index(i,j)] -= 1/hx**2
            A[v_index(i,j),v_index(i,j-1)] = 1/hx**2
            
            b[v_index(i,j)] = rho*g*np.cos(alpha)
            
        # Add continuity equations to base nodes
        if node in node_indices[-1,:]:
            print_info(node, 3*node+2)
            A[p_index(i,j),u_index(i,j+1)] = 1/hx
            A[p_index(i,j),u_index(i,j)] = -1/hx
            A[p_index(i,j),v_index(i,j)] = 1/hy
            # A[p_index(i,j),v_index(i+1,j)] = -1/hy
            
            b[p_index(i,j)] = 0

In [ ]:
# Visualize A
# plt.spy(A, markersize=1)
# np.savetxt("output/A.csv", A, delimiter=",")

In [ ]:
# Solve the system Ax=b
solution = np.linalg.solve(A, b)

In [ ]:
# Extract values of u, v, and p from the solution
u_indices = 3*np.arange((nx-1)*(ny-1))
v_indices = u_indices+1
p_indices = u_indices+2

u_values = np.take(solution, u_indices)
v_values = np.take(solution, v_indices)
p_values = np.take(solution, p_indices)

In [ ]:
# Compute the averages at the center of each cell
U_values = np.zeros((ny-1, nx))
U_values[:,:-1] = u_values.reshape(ny-1, nx-1, order="F")
U_values[:,-1] = U_values[:,0] # Add first column to last one
U_values = (U_values[:,0:-1]+U_values[:,1:])/2

# Compute average of v at the center of each cell (ignore ghost points, i.e. first row)
V_values = np.zeros((ny, nx-1))
V_values[0:-1,:] = v_values.reshape(ny-1, nx-1, order="F")
V_values = (V_values[0:-1,:]+V_values[1:,:])/2

# Obtain pressure at the center of each cell
P_values = p_values.reshape(ny-1, nx-1, order="F")

In [ ]:
# Extract values of u on ice surface
u_surface = U_values[0,:]

In [ ]:
# Setup a meshgrid for plotting
x = np.linspace(hx/2, length-hx/2, nx-1)
y = np.linspace(hy/2, height-hy/2, ny-1)

In [ ]:
# Plot results
fig, axs = plt.subplots(3, 1, sharex=True, constrained_layout=True)

vmin = np.min((U_values, V_values))
vmax = np.max((U_values, V_values))

for ax, values, title in ((axs[0], np.flipud(U_values), "U"), 
                          (axs[1], np.flipud(V_values), "V")):
    im0 = ax.pcolor(x, y, values, vmin=vmin, vmax=vmax, cmap=cmo.deep)
    ax.set_title(title)
    ax.set_ylabel("y", rotation=0)
    ax.set_xlim([0, length])
    ax.set_ylim([0, height])

fig.colorbar(im0, ax=axs[:2], aspect=2*10, pad=-10)

ax = axs[2]
im1 = ax.pcolor(x, y, np.flipud(P_values), cmap=cmo.deep)
ax.set_title("P")
ax.set_ylabel("y", rotation=0)
ax.set_xlabel("x")
ax.set_xlim([0, length])
ax.set_ylim([0, height])

fig.colorbar(im1, ax=axs[2], aspect=8.7)

In [ ]:
# # Plot results
# fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True)
# fig.subplots_adjust(hspace=0.5)

# cax1 = ax1.pcolor(x, y, np.flipud(U_values), cmap=cmo.ice)
# fig.colorbar(cax1, ax=ax1)
# ax1.set_ylabel("y", rotation=0)
# ax1.set_title("U")
# ax1.set_xlim([0, length])
# ax1.set_ylim([0, height])

# cax2 = ax2.pcolor(x, y, np.flipud(V_values), cmap=cmo.ice)
# fig.colorbar(cax2, ax=ax2)
# ax2.set_ylabel("y", rotation=0)
# ax2.set_title("V")
# ax2.set_xlim([0, length])
# ax2.set_ylim([0, height])

# cax3 = ax3.pcolor(x, y, np.flipud(P_values), cmap=cmo.ice)
# fig.colorbar(cax3, ax=ax3)
# ax3.set_xlabel("x")
# ax3.set_ylabel("y", rotation=0)
# ax3.set_title("P")
# ax3.set_xlim([0, length])
# ax3.set_ylim([0, height])

In [ ]:
# Plot the vector field
X, Y = np.meshgrid(x, y)
plt.quiver(X, Y, np.flipud(U_values), np.flipud(V_values))

In [ ]:
# Plot surface velocity
plt.plot(x, u_surface)
plt.ylim([np.min(u_surface), np.max(u_surface)])

# References

[1] Granzow, G. (2014). A tutorial on adjoint methods and their use for data assimilation in glaciology. Journal of Glaciology, 60(221), 440-446. doi:10.3189/2014JoG13J205

[2] Becker, T. W. and B. J. P. Kaus (2018). Numerical Modeling of Earth Systems. An introduction to computational methods with focus on solid Earth applications of continuum mechanics. Lecture notes for USC GEOL557, v. 1.2.1, available online at http://www-udc.ig.utexas.edu/external/becker/Geodynamics557.pdf, accessed 01/2018.